In [1]:
import pandas as pd
import numpy as np
from uszipcode import SearchEngine, SimpleZipcode, Zipcode

search = SearchEngine()
zipcode = search.by_zipcode(10030)
zipcode.zipcode # access attributes
zipcode.state

'NY'

In [2]:
zip_columns = ["DC_ZIPCODE"] + ["Monthly_Top_" + str(i) + "_Customer_Zip" for i in range(1, 11)]

### Build a map of zipcodes to state

In [3]:
train = pd.read_csv("data/X_train.csv", index_col=0)

c:\users\patty\appdata\local\programs\python\python37-32\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
test = pd.read_csv("data/X_test.csv", index_col=0)

In [14]:
vehicles = list()
for chunk in pd.read_csv("data/vehicle_registration_data_2018.csv", index_col=0, chunksize=10 ** 5):
    vehicles.append(set(chunk["ZIP_CODE"].unique()))

In [ ]:
train.head()[zip_columns]

In [15]:
vehicles

[{501,
  1001,
  1002,
  1003,
  1004,
  1005,
  1007,
  1008,
  1009,
  1010,
  1011,
  1012,
  1013,
  1014,
  1020,
  1021,
  1022,
  1026,
  1027,
  1028,
  1029,
  1030,
  1031,
  1032,
  1033,
  1034,
  1035,
  1036,
  1037,
  1038,
  1039,
  1040,
  1041,
  1050,
  1053,
  1054,
  1056,
  1057,
  1059,
  1060,
  1061,
  1062,
  1063,
  1066,
  1068,
  1069,
  1070,
  1071,
  1072,
  1073,
  1074,
  1075,
  1077,
  1079,
  1080,
  1081,
  1082,
  1083,
  1084,
  1085,
  1086,
  1088,
  1089,
  1090,
  1092,
  1093,
  1094,
  1095,
  1096,
  1097,
  1098,
  1101,
  1102,
  1103,
  1104,
  1105,
  1106,
  1107,
  1108,
  1109,
  1111,
  1115,
  1116,
  1118,
  1119,
  1128,
  1129,
  1138,
  1139,
  1144,
  1151,
  1199,
  1201,
  1202,
  1220,
  1222,
  1223,
  1224,
  1225,
  1226,
  1227,
  1229,
  1230,
  1235,
  1236,
  1237,
  1238,
  1240,
  1242,
  1243,
  1244,
  1245,
  1247,
  1252,
  1253,
  1254,
  1255,
  1256,
  1257,
  1258,
  1259,
  1260,
  1262,
  1263,
  1264,
 

In [16]:
zipsets = [set(train[column]) for column in zip_columns] + [set(test[column]) for column in zip_columns] + vehicles 
all_zips = { int(code) for codes in zipsets for code in codes if not np.isnan(code) and 601 <= code <= 99950 }
zip_states = { code: search.by_zipcode(code).state for code in all_zips }

In [17]:
len(zip_states)

39677

In [18]:
data = {'zip': list(zip_states.keys()), 'state': list(zip_states.values())}
zip_df = pd.DataFrame.from_dict(data)

In [19]:
zip_df.to_csv("zip_states.csv")
zip_df

,zip,state
0,1001,MA
1,1002,MA
2,1003,MA
3,1004,MA
4,1005,MA
5,1007,MA
6,1008,MA
7,1009,MA
8,1010,MA
9,1011,MA


### Augment the data with states

In [ ]:
chunk = pd.read_csv("data/train_augmented_2.csv", index_col=0, nrows=200)
chunk["DC_STATE"] = chunk["DC_ZIPCODE"].astype('float64').map(lambda code: zip_states.get(code) or np.nan)
for column in zip_columns[1:]:
    chunk[column] = chunk[column].astype('float64').map(lambda code: zip_states.get(code) or np.nan)
chunk[zip_columns]

In [ ]:
out_file = "data/train_augmented_3.csv"
open(out_file, 'w').close()

with open(out_file, 'a') as f:
    writeHeader = True
    for chunk in pd.read_csv("data/train_augmented_2.csv", index_col=0, chunksize=10 ** 5):
        chunk["DC_STATE"] = chunk["DC_ZIPCODE"].astype('float64').map(lambda code: zip_states.get(code) or np.nan)
        for column in zip_columns[1:]:
            chunk[column] = chunk[column].astype('float64').map(lambda code: zip_states.get(code) or np.nan)
        chunk.to_csv(f, header=writeHeader)
        writeHeader = False